# 1. Data Preparation
Before starting to train the model, some steps need to be done.

# 1.1. ROI Extraction
The images are large in size (above 2000 x 2000 pixels), so it is better to extract the area that is important to us by using a cropping function. This cropping process should be applied to both the image and the respective masks.

<div style="text-align: center;">
    <img src="Images/image-11.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>


* TASK:

In [7]:
import cv2
import os
from pathlib import Path
from PIL import Image

# Input and output directories
raw_imgs = Path("../images/raw_images").glob("*.jpg")
raw_labels = Path("../images/raw_labels").glob("*.png")
cropped_img_dir = Path("../images/") / "cropped_images"
cropped_label_dir = Path("../images/") / "cropped_labels"
cropped_img_dir.mkdir(exist_ok=True)
cropped_label_dir.mkdir(exist_ok=True)

def crop_imgs(input_imgs, output_dir) -> None:
   for img_path in input_imgs:
      #crop the center section ~> 1400W * 1840H
      with Image.open(img_path) as img:
         width, height = img.size
         left = (width - 1400) // 2
         top = (height - 1840) // 2
         right = left + 1400
         bottom = top + 1840
         cropped = img.crop((left, top, right, bottom))
         cropped.save(output_dir / img_path.name)
   return
   
crop_imgs(raw_imgs, cropped_img_dir)
crop_imgs(raw_labels, cropped_label_dir)

# 1.2. Image Format Converting 
It is recommended to convert the format of both the image and mask to TIFF format, which is suitable for the recommended Convolutional Neural Network (CNN) model.

* TASK:

In [8]:
### Code
##ConTif
cropped_imgs = Path("../images/cropped_images").glob("*.jpg")
cropped_labels = Path("../images/cropped_labels").glob("*.png")
tif_img_dir = Path("../images/") / "tif_images"
tif_label_dir = Path("../images/") / "tif_labels"
tif_img_dir.mkdir(exist_ok=True)
tif_label_dir.mkdir(exist_ok=True)

def convert_imgs_to_tif(input_imgs, output_dir) -> None:
    for img_path in input_imgs:
        # Convert it to Tif
        with Image.open(img_path) as img:
            tif_name = img_path.stem + ".tif"
            img.save(output_dir / tif_name, format="TIFF")
    return

convert_imgs_to_tif(cropped_imgs, tif_img_dir)
convert_imgs_to_tif(cropped_labels, tif_label_dir)

# 1.3. Image Augmentation

After extracting the ROI and converting the image and mask to TIFF format, we need to increase our dataset size using augmentation techniques. Here's how you can implement this method, ensuring that the image name and the respective mask name are the same:

* Define Augmentation Parameters: Determine the augmentation techniques to apply, such as rotation, flipping, scaling, etc.

* Loop Through Images: Iterate through each image and its corresponding mask.

* Apply Augmentation: Apply the defined augmentation techniques to both the image and its mask.

* Save Augmented Images: Save the augmented images and their masks with the same names as the original images and masks.

<div style="text-align: center;">
    <img src="Images/image-12.png" alt="Alt text" style="display: block; margin: 0 auto;">
</div>

* TASK:

In [13]:
import numpy as np
import random
from scipy.ndimage import rotate, shift

# Define functions for each operation

def rotation(image, seed):
    random.seed(seed)
    angle = random.uniform(-25, 25)  # Rotate between -25 and 25 degrees
    r_img = rotate(image, angle, reshape=False, mode='reflect')
    return r_img

def h_flip(image, seed):
    random.seed(seed)
    if random.random() > 0.5:
        hflipped_img = np.fliplr(image)
    else:
        hflipped_img = image
    return hflipped_img

def v_flip(image, seed):
    random.seed(seed)
    if random.random() > 0.5:
        vflipped_img = np.flipud(image)
    else:
        vflipped_img = image
    return vflipped_img

def v_transl(image, seed):
    random.seed(seed)
    pixels = random.randint(-30, 30)
    shift_vals = [pixels if i == 0 else 0 for i in range(image.ndim)]
    return shift(image, shift=shift_vals, mode='reflect')

def h_transl(image, seed):
    random.seed(seed)
    pixels = random.randint(-30, 30)
    shift_vals = [pixels if i == 1 else 0 for i in range(image.ndim)]
    return shift(image, shift=shift_vals, mode='reflect')

* TASK:

In [ ]:
###Use the functions to implement the augmentation for both images and masks
transformations = {
    'rot': rotation,
    'hflip': h_flip,
    'vflip': v_flip,
    'vtrans': v_transl,
    'htrans': h_transl
}
tif_imgs = Path("../images/tif_images").glob("*.tif")
tif_labels = Path("../images/tif_labels").glob("*.tif")
augmented_img_dir = Path("../images/") / "augmented_images"
augmented_label_dir = Path("../images/") / "augmented_labels"
augmented_img_dir.mkdir(exist_ok=True)
augmented_label_dir.mkdir(exist_ok=True)

for img_path, label_path in zip(tif_imgs, tif_labels):
    img = np.array(Image.open(img_path))
    label = np.array(Image.open(label_path))

    for t_name, t_func in transformations.items():
        seed = random.randint(0, 10000)  # Same seed for image and mask

        aug_img = t_func(img, seed)
        aug_msk = t_func(label, seed)

        # Save augmented versions
        aug_img_pil = Image.fromarray(aug_img.astype(np.uint8))
        aug_label_pil = Image.fromarray(aug_msk.astype(np.uint8))
        aug_img_pil.save(augmented_img_dir / f"{t_name}_{img_path.name}")
        aug_label_pil.save(augmented_label_dir / f"{t_name}_{label_path.name}")

3485
4905
3401
6171
1761
6922
8584
2000
7773
7881
5375
1166
9116
2308
599
544
4579
2341
7078
4240
1018
6465
5525
2928
5693
1329
2515
3545
6925
6752
9237
7365
6529
2332
7274
7229
4066
2022
7251
7555
7677
915
5187
9136
4896
6013
2830
6995
6695
9687
1144
7886
9335
6604
5185
9900
8818
8868
6331
9774
8462
9556
4884
3500
8890
9638
6059
1399
4080
8142
5769
2877
2730
6711
3822
2151
7476
5286
7735
8010
1551
2474
6540
1184
2757
5992
2813
9807
5710
8894
6223
9563
8405
3878
4647
7011
5829
8480
8019
6153
973
800
9691
8017
4616
1056
6055
9994
6847
943
3894
9806
9157
4303
4711
8033
8456
8811
9011
6131
8051
7447
6504
2144
6759
3583
4275
2219
3604
8650
4401
2329
8119
3405
3255
1982
1282
3233
2173
3605
5311
1068
3391
6393
5307
5443
3290
8085
9976
19
708
5727
6375
5262
1166
913
7823
2867
3292
985
3583
4275
2219
3604
8650
4401
2329
8119
3405
3255
1982
1282
3233
2173
3605
